In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Coalesce").getOrCreate()
spark

c:\Users\kumav\Desktop\Spark_Project\youtube_de_project1-master\data_Phoenix_migration\venv\lib\site-packages\pyspark\context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [2]:
data = [
 ("Sales", 5000, "John"),
 ("Sales", 6000, "Doe"),
 ("HR", 7000, "Jane"),
 ("HR", 8000, "Alice"),
 ("IT", 4500, "Bob"),
 ("IT", 5500, "Charlie"),
]

columns = ['Department', 'salary', 'emp_name']

df = spark.createDataFrame(data, schema=columns)
df.show()

+----------+------+--------+
|Department|salary|emp_name|
+----------+------+--------+
|     Sales|  5000|    John|
|     Sales|  6000|     Doe|
|        HR|  7000|    Jane|
|        HR|  8000|   Alice|
|        IT|  4500|     Bob|
|        IT|  5500| Charlie|
+----------+------+--------+



In [3]:
print("Initial Number of partitions:", df.rdd.getNumPartitions())

Initial Number of partitions: 12


In [4]:
coalesce1 = df.coalesce(4)
print("Initial Number of partitions:", coalesce1.rdd.getNumPartitions())

Initial Number of partitions: 4


In [9]:
reparti = df.repartition(34)
print("Initial Number of partitions:", reparti.rdd.getNumPartitions())

Initial Number of partitions: 34


In [10]:
data2 = [
(1, "2024-01-01", 200),
(1, "2024-01-02", 150),
(2, "2024-01-01", 300),
(3, "2024-01-01", 100),
(1, "2024-01-03", 250),
(3, "2024-01-02", 200),
(2, "2024-01-02", 100),
(2, "2024-01-03", 200)
]
column2 = [ "Customer_id", "Transaction_date", "amount"]

df1 = spark.createDataFrame(data2, column2)
df1.show()

+-----------+----------------+------+
|Customer_id|Transaction_date|amount|
+-----------+----------------+------+
|          1|      2024-01-01|   200|
|          1|      2024-01-02|   150|
|          2|      2024-01-01|   300|
|          3|      2024-01-01|   100|
|          1|      2024-01-03|   250|
|          3|      2024-01-02|   200|
|          2|      2024-01-02|   100|
|          2|      2024-01-03|   200|
+-----------+----------------+------+



In [16]:
from pyspark.sql.functions import sum, avg, count, col

aggregateed_df = df1.groupBy("Customer_id").agg(
    sum("amount").alias("TotalAmount"),
    avg("amount").alias("Avg_amount"),
    count("amount").alias("Num_Transaction")
)
aggregateed_df.show()

+-----------+-----------+----------+---------------+
|Customer_id|TotalAmount|Avg_amount|Num_Transaction|
+-----------+-----------+----------+---------------+
|          1|        600|     200.0|              3|
|          2|        600|     200.0|              3|
|          3|        300|     150.0|              2|
+-----------+-----------+----------+---------------+



In [17]:
filtered_df = aggregateed_df.filter(col("Num_Transaction") < 3)
filtered_df.show()

+-----------+-----------+----------+---------------+
|Customer_id|TotalAmount|Avg_amount|Num_Transaction|
+-----------+-----------+----------+---------------+
|          3|        300|     150.0|              2|
+-----------+-----------+----------+---------------+



In [26]:
# # Create a global temporary view
# aggregateed_df.createGlobalTempView("Emp")

# Query using the global_temp database
result = spark.sql("SELECT * FROM global_temp.Emp")

# Show results
result.show()


+-----------+-----------+----------+---------------+
|Customer_id|TotalAmount|Avg_amount|Num_Transaction|
+-----------+-----------+----------+---------------+
|          1|        600|     200.0|              3|
|          2|        600|     200.0|              3|
|          3|        300|     150.0|              2|
+-----------+-----------+----------+---------------+



----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 54246)
Traceback (most recent call last):
  File "C:\Users\kumav\AppData\Local\Programs\Python\Python36-32\lib\socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\kumav\AppData\Local\Programs\Python\Python36-32\lib\socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\kumav\AppData\Local\Programs\Python\Python36-32\lib\socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\kumav\AppData\Local\Programs\Python\Python36-32\lib\socketserver.py", line 724, in __init__
    self.handle()
  File "c:\Users\kumav\Desktop\Spark_Project\youtube_de_project1-master\data_Phoenix_migration\venv\lib\site-packages\pyspark\accumulators.py", line 262, in handle
    poll(accum_updates)
  File "c:\Users\kuma